In [8]:
import pandas as pd

from trampact.preprocess import get_data

In [9]:
!pwd

/home/primitive/code/eangelid/trampact/notebooks


In [10]:
IRIS_T1_CSV = 'iris_T1.csv'
IRIS_T2_CSV = 'iris_T2.csv'
BD_GENT_2006 = 'BD_GENT_2006.csv'
BD_GENT_2016 = 'BD_GENT_2016.csv'

#### Load T1 and T2 IRIS codes

In [11]:
iris_T1 = get_data(IRIS_T1_CSV, is_local_csv=True)
iris_T1.head()

/home/primitive/code/eangelid/trampact/trampact


,iris_id,NOM_IRIS
0,60881304,Saint-Sylvestre
1,60881503,Jeanne d'Arc-Fuon-Cauda
2,60881303,Gorbella
3,60881505,Michelet
4,60881302,Saint-Barthelemy


In [12]:
iris_T2 = get_data(IRIS_T2_CSV, is_local_csv=True)[['iris_id', 'NOM_IRIS']]
iris_T2.head()

/home/primitive/code/eangelid/trampact/trampact


,iris_id,NOM_IRIS
0,61230107,Centre Ville
1,61230103,Les Pomarels
2,61230102,Les Paluds
3,61230101,Le Lac
4,60883102,Parc des Sports


#### Load demographic data

In [13]:
bd_gent_2006 = get_data(BD_GENT_2006, is_local_csv=True)
print(bd_gent_2006.shape)
bd_gent_2006.head()

/home/primitive/code/eangelid/trampact/trampact
(500, 45)


,iris_id,t_actifs_2006,t_chom_2006,empl_2006,ouvrier_2006,TP_2006,Int_2006,app_stage_2006,cdd_2006,rev_2006,...,tx_TC_2006,tx_HH_moins2ans_2006,tx_HH_2_4ans_2006,tx_HH_5_9ans_2006,tx_HH_plus10ans_2006,tx_HH_with_park_2006,tx_HH_with_voit_2006,tx_HH_1voit_2006,tx_HH_2voit_2006,tx_empl_prec_2006
0,60010000,32.377778,2.088889,8.355556,0.000000,6.266667,0.000000,0.000000,8.355556,17465.0,...,12.903226,12.903226,29.032258,22.580645,83.870969,80.645160,122.580645,74.193547,48.387097,25.806453
1,60020000,12.008551,3.002138,4.002850,4.002850,1.000713,0.000000,0.973333,0.000000,NaN,...,0.000000,25.000002,33.333331,16.666665,58.333333,66.666669,133.333331,108.333328,25.000002,8.105335
2,60030000,232.021978,24.725275,51.282051,80.586081,32.967033,0.915751,3.622305,28.388278,13866.0,...,0.000000,24.111964,22.533232,20.918190,28.058792,58.413059,86.544473,56.081273,30.463200,14.191041
3,60040101,558.359852,79.552998,189.506664,142.278837,104.191062,0.000000,9.724609,43.918480,20108.0,...,12.450518,3.569231,13.748829,9.705411,36.637534,48.238210,61.290878,30.845691,30.445187,9.607261
4,60040102,1322.676650,95.706717,453.856986,188.318276,203.919192,11.170277,17.151069,81.122892,18178.0,...,5.740664,5.073478,11.192460,16.480261,42.846445,52.077781,70.229033,33.934914,36.294119,8.274451


In [14]:
bd_gent_2016 = get_data(BD_GENT_2016, is_local_csv=True)
print(bd_gent_2016.shape)
bd_gent_2016.head()

/home/primitive/code/eangelid/trampact/trampact
(500, 45)


,iris_id,t_actifs_2016,t_chom_2016,empl_2016,ouvrier_2016,TP_2016,Int_2016,app_stage_2016,CDD_2016,rev_2016,...,tx_TC_2016,tx_HH_moins2ans_2016,tx_HH_2_4ans_2016,tx_HH_5_9ans_2016,tx_HH_plus10ans_2016,tx_HH_with_park_2016,tx_HH_with_voit_2016,tx_HH_1voit_2016,tx_HH_2voit_2016,tx_empl_prec_2016
0,60010000,36.365591,5.741935,4.784946,0.000000,6.698925,0.000000,0.000000,1.913978,NaN,...,26.315789,28.947368,28.947368,21.052632,47.368421,73.684211,110.526316,73.684211,36.842105,5.263158
1,60020000,31.146667,5.840000,4.866667,4.866667,6.813333,0.000000,0.973333,5.840000,NaN,...,0.000000,15.625000,25.000000,12.500000,68.750000,34.375000,90.625000,65.625000,25.000000,21.875000
2,60030000,269.050007,43.467660,95.085507,67.918219,42.562084,1.811153,3.622305,18.111525,15670.0,...,0.000000,11.443818,23.560802,24.268852,41.063111,69.707541,93.941508,52.205231,41.736277,8.751155
3,60040101,604.818909,96.239886,183.644202,102.723350,53.893508,3.788325,9.724609,40.522157,21136.0,...,9.189947,13.618267,38.343338,33.730176,18.561670,75.119975,80.093156,55.589045,24.504111,8.934094
4,60040102,1265.456645,91.799759,341.483429,157.342286,152.709438,1.142076,17.151069,81.025567,24220.0,...,5.396948,6.232465,14.154485,8.492861,52.291805,58.982592,76.012828,39.134568,36.878260,7.848448


#### Filter tram quarters only

In [20]:
bd_gent_2006_T1 = bd_gent_2006.merge(iris_T1, on='iris_id').drop('NOM_IRIS', axis=1)
print(bd_gent_2006_T1.shape)

(35, 45)


In [21]:
bd_gent_2016_T1 = bd_gent_2016.merge(iris_T1, on='iris_id').drop('NOM_IRIS', axis=1)
print(bd_gent_2016_T1.shape)

(35, 45)


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
draw_feature = 'tx_empl_prec'
bins=20

fig, axs = plt.subplots(2,2, figsize=(14,14), squeeze=True)
axs[0].hist(bd_gent_2006[f'{draw_feature}_2006'], bins=bins)
axs[1].hist(bd_gent_2016[f'{draw_feature}_2016'], bins=bins)
axs[2].hist(bd_gent_2006_T1[f'{draw_feature}_2006'], bins=bins)
axs[3].hist(bd_gent_2016_T1[f'{draw_feature}_2016'], bins=bins)

# axs[0][0].hist(bd_gent_2006[f'{draw_feature}_2006'], bins=bins)
# axs[0][1].hist(bd_gent_2016[f'{draw_feature}_2016'], bins=bins)
# axs[1][0].hist(bd_gent_2006_T1[f'{draw_feature}_2006'], bins=bins)
# axs[1][1].hist(bd_gent_2016_T1[f'{draw_feature}_2016'], bins=bins)

ax[0].set_title('2006')
ax[0].set_ylabel('Occurences')
ax[0].set_xlabel(draw_feature + ' (Nice)')

ax[1].set_title('2016')
ax[1].set_ylabel('Occurences')
ax[1].set_xlabel(draw_feature + ' (Nice)')

ax[2].set_title('2006')
ax[2].set_ylabel('Occurences')
ax[2].set_xlabel(draw_feature + ' (T1)')

ax[3].set_title('2016')
ax[3].set_ylabel('Occurences')
ax[3].set_xlabel(draw_feature + ' (T1)')

fig.suptitle(draw_feature)
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'hist'